# Sistemas de recomendación: Filtros colaborativos
M4U2 - Ejercicio 1

## ¿Qué vamos a hacer?
- Explorar el planteamiento de los filtros colaborativos
- Crear un dataset a resolver por sistemas de recomendación
- Implementar la función de coste y descenso de gradiente
- Entrenar un modelo de recomendación por filtros colaborativos
- Realizar predicciones de recomendaciones
- Reentrenar el modelo incorporando nuevas valoraciones
- Recomendar ejemplos similares a otros

In [ ]:
# TODO: Usa esta celda para importar todas las librerías necesarias

import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance

np.random.seed(42)

# Crear el dataset sintético

Un ejemplo habitual son las recomendaciones de películas en un portal de streaming de video. En este caso, un dataset tendría estas características p. ej.:
- *m*: Nº de películas.
- *n*: Nº de características de cada película y de coeficientes de cada usuario para las mismas.
- $n_u$: Nº de usuarios del portal.
- $n_ru$ y $n_r$: Porcentaje de valoraciones de cada película y nº de valoraciones total, conocidos de antemano.
- *X*: Matriz 2D de características de cada película, tamaño (nº de películas, nº de características).
- $\Theta$: Matriz 2D de coeficientes de cada usuario para cada película, tamaño (nº de características, nº de usuarios).
- *Y*: Matrix 2D de valoraciones de cada usuario para cada película, tamaño (nº de películas, nº de usuarios).

Vamos a crear un dataset sintético como habitualmente, pero ésta vez enfocado a los sistemas de recomendación, con algunas diferencias frente al de regresión lineal:
- Las características predictoras o independientes *X* (tamaño (*m*, *n* + 1)) que representan las características de cada ejemplo, **no son conocidas de antemano**.
- El vector $\Theta$ (*Theta*) es 2D (tamaño (*n* + 1, $n_u$)), puesto que ahora representa los coeficientes de las características para cada usuario. De nuevo, **no es conocido de antemano**.
- El vector *Y* es 2D (tamaño (*m*, $n_u$)), puesto que ahora representa la valoración de cada usuario para cada ejemplo.
- El vector *Y* contendrá tanto las valoraciones "reales" que ha emitido cada usuario para cada película que ha valorado, como, al final del entrenamiento, sus valoraciones predichas para recomendarle una película u otra.
- *R* será una matriz "máscara" sobre *Y*, utilizada para indicar qué valoraciones de *Y* son reales y emitidas por un usuario, y por tanto sólo aquellas utilizadas para entrenar el modelo.

Para tener a mano, os dejamos esta tabla rápida para consultar el tamaño de cada matriz:
- $X (m, n + 1)$
- $\Theta (n + 1, n_u)$
- $Y (m, n_u)$

Para no complicar más la implementación, en este caso no preprocesaremos los datos.

Sigue las instrucciones para generar un dataset con las características necesarias para poder resolverlo por un filtro colaborativo:

In [ ]:
# TODO: Crea un dataset con las características necesarias para un sistema de recomendación
# Recuerda que puedes volver a esta celda y modificar las características del dataset en cualquier momento

m = 1000    # Nº de ejemplos
n = 4    # Nº de características de cada ejemplo/usuario
n_u = 100    # Nº de usuarios
n_rr = 0.25    # Porcentaje de valoraciones conocidas de antemano

# Crea una X con valores aleatorios y tamaño (m, n)
# Inserta una columna de 1. en la primera columna
X_verd = [...]

# Crea una Theta_verd con valores aleatorios y tamaño (n + 1, n_u)
Theta_verd = [...]

# Crea una Y_verd de tamaño (m, n_u) multiplicando X_verd y Theta_verd transpuesta
Y_verd = [...]

# Crea una matriz R de 0s con tamaño (m, n_u)
r = [...]
count_r = round(n_rr * r.size)    # nº de valoraciones conocidas o 1s en R
while count_r:
    # Genera un int aleatorio entre [0, m] como índice de R
    i = [...]
    # Genera un int aleatorio entre [0, n_u] como índice de R
    j = [...]
    
    # Cambia dicho elemento de R a 1. si no se ha cambiado antes y resta 1 al nº de valoraciones conocidas
    if not r[i, j]:
        r[i, j] = 1.

        count_r -= 1

# Cuenta los valores de R que no sean 0.
n_r = [...]

# Genera una Y con sólo las valoraciones conocidas usando R
y = [...]

print('Tamaño de X(m, n+1), Theta(n+1, n_u) e Y(m, n_u) verdaderos:')
print(X_verd.shape, Theta_verd.shape, Y_verd.shape)
print('Tamaño de y y R conocidas de antemano:')
print(y.shape, r.shape)
print('Nº de elementos de R o valoraciones conocidas:', n_r)

# Función de coste y descenso de gradiente

Vamos a implementar la función de coste y el descenso de gradiente regularizados entrenar el modelo de ML.

Conceptualmente, vamos a seguir unos pasos diferentes a los de la regresión lineal:

Mientras que en la regresión lineal eran conocidas *Y* y *X* y podíamos optimizar iterativamente $\Theta$ para reducir el coste, en esta ocasión *X* tampoco es conocida de antemano, ya que habitualmente es imposible en la práctica conocer o tener anotadas de antemano todas las características de todos los ejemplos o películas.

Además, mientras que sí tenemos algunas valoraciones por cada usuario de algunas películas, solemos tener un porcentaje bastante bajo de valoraciones para cada ejemplo, por lo que *Y* no es completamente conocida de antemano sino que la mayoría de sus valores estarán vacíos inicialmente.

Nuestro objetivo pues no será resolver $\Theta$ sino *Y* para rellenarla obteniendo todas las valoraciones predichas de cada usuario para cada ejemplo.

Por tanto, el algoritmo de entrenamiento será:
1. Recopilamos los ejemplos y las valoraciones en las matrices *X*, $\Theta$ e *Y*.
1. Marcamos las valoraciones conocidas en la matriz dispersa *R*.
1. Dadas *X* e *Y*, podemos obtener $\Theta$.
1. Dadas $\Theta$ e *Y*, podemos obtener *X*.
1. Estimamos de forma iterativa *X* y $\Theta$ en cada iteración hasta que el entrenamiento converja en un coste mínimo.
1. Cuando dispongamos de más valoraciones, reentrenamos el modelo añadiéndolas a *Y* y marcándolas en *R*.

En la siguiente celda, sigue las instrucciones para implementar la función de coste y gradient descent regularizados para un filtro colaborativo, siguiendo las siguientes fórmulas:

$$ \min\limits_{\theta^0, ..., \theta^{n_u}, x^0, ..., x^{n_m}} J(x^0, ..., x^{n_m}, \theta^0, ..., \theta^{n_u}) = \min\limits_{\theta^0, ..., \theta^{n_u}, x^0, ..., x^{n_m}} [\frac{1}{2} \sum\limits_{(i, k): r(i, k)=1} (x^i \times \theta^T_k - y^i_k)^2 $$
$$ + \frac{\lambda}{2} \sum\limits_{i=0}^{n} \sum\limits_{k=0}^{n_u} (x^i_k)^2 + \frac{\lambda}{2} \sum\limits_{j=0}^{n} \sum\limits_{k=0}^{n_u} (\theta^j_k)^2] $$
$$ x^i_k := x^i_k - \alpha (\sum\limits_{j: r(i, k) = 1} (x^i \times \theta^T_k - y^i_k) \theta^j_k + \lambda x^i_k); $$
$$ \theta^j_k := \theta^j_k - \alpha (\sum\limits_{i: r(i, k) = 1} (x^i \times \theta^{j T}  - y^i_k) x^i_k + \lambda \theta^j_k); \space j = 0 \rightarrow \lambda = 0 $$

In [ ]:
# TODO: Implementa la función de coste para filtros colaborativos

def cost_function_collaborative_filtering_regularized(x, theta, y, r, lambda_=0.):
    # CONSEJOS: Plantea las operaciones paso a paso en papel, apuntando las dimensiones de los vectores originales y las del resultado de cada operación intermedia
    # Utiliza ndarray.reshape() si lo necesitas, especialmente con vectores 1D (p. ej. (6,)) que pueden darte resultados no esperados en Numpy
    # Usa m, n, n_u en ndarray.reshape(), no valores "hard-coded" como 6, 20, etc.
    # Utiliza np.matmul() para multiplicar matrices
    # Para entrenar sólo sobre valores conocidos, multiplica R por el resultado de la resta de la hipótesis e y como matriz de máscara
    # Escogiendo los slices o vectores de X, Theta e Y correctamente, no hay gran diferencia con la regresión lineal
    j = [...]

    # Calcula el factor de regularización para X
    x_reg = [...]

    # Calcula el factor de regularización para Theta
    # Recuerda no regularizar la primera columna
    theta_reg = [...]

    j = [...]

    return j

### Comprobación de la implementación de la función de coste

Comprueba tu implementación de la función de coste en las siguientes circunstancias:
1. Si `theta = Theta_verd`, `j = 0`
1. Si `theta = Theta_verd` y `lambda_ != 0`, `j != 0`
1. Cuanto más se aleja `lambda_` de 0, a igual `theta`, más aumenta `j`
1. Si todos los elementos de `r` son 0, no se considera ningún elemento para el entrenamiento, por lo tanto `j = 0`.

In [ ]:
# TODO: Comprueba la implementación de la función de coste

Anota tus resultados en esta celda:
1. Experimento 1
1. Experimento 2
1. Experimento 3
1. Experimento 4

In [ ]:
# TODO: Implementa el entrenamiento por descenso de gradiente para filtros colaborativos

def gradient_descent_collaborative_filtering_regularized(x, theta, y, r, lambda_=0., alpha=1e-3, n_iter=1e3, e=1e-3):
    # Para entrenar sólo sobre valores conocidos, multiplica R por el resultado de la resta de la hipótesis e y como matriz de máscara

    n_iter = int(n_iter)    # Convierte n_iter a int para poder usarlo en range()
    
    # Inicializa j_hist con el historial de valores de la función de coste
    j_hist = []
    # Añade como primer valor el coste de la función de coste para los valores iniciales
    j_hist.append(cost_function_collaborative_filtering_regularized([...]))
    
    for iter_ in range(n_iter):
        # Inicializa unas theta y x vacías para rellenar con el gradiente con ndarrays del mismo tamaño de los originales
        # y valores de vector vacío (más optimizado), zeros o aleatorios, y así no modificar theta, que debe mantenerse constante durante la iteración iter_
        theta_grad = [...]
        x_grad = [...]
                
        for k in range(n_u):
            # Calcula el gradiente para actualizar theta en esta iteración
            # Utiliza theta y no theta_grad en las operaciones intermedias, ya que queremos modificar theta_grad y no la theta original
            theta_grad[:, k] = [...]
            
            # Para toda theta_grad, excepto la primera columna, añade el término de regularización
            theta_grad[1:, k] += [...]
            
        for i in range(m):
            # Calcula el gradiente para actualizar X en esta iteración
            # Sigue pasos similares al gradiente de theta para implementar la función correspondiente
            # Suma el término de regularización
            x_grad[i, :] = [...]

        # Actualiza X y Theta con los gradientes
        x -= alpha * x_grad
        theta -= alpha * theta_grad
        
        # Si lo necesitas, comprueba cómo se van actualizando X y Theta
        #print('\nValores de X y Theta actualizados:')
        #print(x)
        #print(x.shape)
        #print(theta)
        #print(theta.shape)
        
        # Calcula el coste en esta iteración y añádelo al historial de costes
        j_cost = cost_function_collaborative_filtering_regularized([...])
        j_hist.append(j_cost)

        # Si no es la primera iteración y la diferencia absoluta entre el coste y el de la iteración anterior es
        # menor que e, declara la convergencia
        if [...]:
            print('Converge en iteración nº', iter_)
            
            break
    else:
        print('Nº máx. de iteraciones {} alcanzado'.format(n_iter))
        
    return j_hist, x, theta

# Entrenamiento del modelo

Una vez implementadas las funciones correspondientes, vamos a entrenar el modelo.

Para ello, completa la siguiente celda de código con pasos equivalentes a otros modelos de ejercicios previos.

In [ ]:
# TODO: Entrena un modelo de sistema de recomendación por filtros colaborativos

# Genera una X y Theta inicial con valores aleatorios y el mismo tamaño que X_verd y Theta_verd
x_init = [...]
theta_init = [...]

alpha = 1e-2
lambda_ = 0.
e = 1e-3
n_iter = 1e4
print('Hiperparámetros usados:')
print('Alpha:', alpha, 'Lambda:', lambda_, 'Error:', e, 'Nº iter', n_iter)

t0 = time.time()
j_hist, x, theta = gradient_descent_collaborative_filtering_regularized([...])
print('Duración del entrenamiento:', time.time() - t0)

print('\nÚltimos 10 valores de la función de coste:')
print(j_hist[-10:])
print('\nError final:')
print(j_hist[-1])

Como hemos hecho en ocasiones previas, representa gráficamente la evolución de la función de coste para comprobar que el entrenamiento del modelo ha sido correcto:

In [ ]:
# TODO: Representa gráficamente la función de coste del entrenamiento del modelo vs el nº de iteraciones
plt.figure()

plt.plot([...])

# Añade un título, etiquetas a ambos ejes de la gráfica y una rejilla
[...]

plt.show()

### Comprobación de la implementación del descenso de gradiente

Comprueba tu implementación del entrenamiento del modelo en las siguientes circunstancias:
1. Si `theta = Theta_verd`, el model converge en 1 o 2 iteraciones con un coste final `j = 0`
1. Cuanto más se aleja `theta` de `Theta_verd`, hay un mayor coste intermedio y más iteraciones hasta que converge el modelo
1. Cuantos más elementos tiene `r`, menos tarda en converger y menos coste final tiene el modelo

In [ ]:
# TODO: Comprueba la implementación del descenso de gradiente

Anota tus resultados en esta celda:
1. Experimento 1
1. Experimento 2
1. Experimento 3

# Realización de predicciones de recomendaciones

Una vez entrenado el modelo, podemos resolver la matriz de recomendaciones *Y*, que contiene como decíamos tanto las valoraciones emitidas por los usuarios como una predicción de la valoración de cada usuario para cada ejemplo.

Recuerda que utilizábamos la matriz *R* para marcar con un 1. las valoraciones reales y con un 0. aquellas que se han predicho y no eran conocidas de antemano.

Para realizar una predicción y recomendar ejemplos a los usuarios (p. ej. películas), sigue las instrucciones para completar la siguiente celda de código:

In [ ]:
# TODO: Realiza predicciones de ejemplos para los usuarios

# Muestra las valoraciones de la matriz Y
print('Valoraciones conocidas de antemano (10 primeras filas y columnas):')
print(y[:10, :10] * r[:10, :10])    # Limita el nº de filas y columnas de Y para mostrarlo por pantalla
# Muestra más o menos filas y columnas si es necesario para validar tu modelo
# En el resultado, un valor de "0." indica un "0." en esa posición en R, o que esa valoración inicial no es conocida

# Calcula las predicciones obtenidas por el modelo a partir de X y Theta
y_pred = [...]

print('\nValoraciones predichas (10 primeras filas y columnas):')
print(y_pred[:10, :10])

# Calcula los residuos de las predicciones
# Recuerda que los residuos son la diferencia en valor absolutos entre el valor real previamente conocido y las predicciones del modelo
# Recuerda calcularlos sólo cuando la valoración inicial es conocida, multiplicando los residuos por R
y_residuo = [...]

print('\nResiduos del modelo (10 primeras filas y columnas):')
print(y_residuo[:10, :10])

# Muestra las predicciones y valoraciones iniciales de un usuario dado
jj = 0    # Escoge un índice de usuario entre 0 y n_u

print('\nValoraciones reales y predichas para el usuario nº {}:'.format(jj + 1))
print(y_pred[:, jj])

# Ordena los índices de los ejemplos que recomendaríamos a cada usuario en función de sus valoraciones, en orden descendente
# Recuerda eliminar de la lista las valoraciones emitidas inicialmente por el usuario, o películas ya vistas, aquellas cuya R[i, k] == 0
# Puedes ordenar un ndarray con numpy.sort()
print('\nValoraciones predichas para el usuario nº {}:'.format(jj + 1))
print([...])

# Puedes obtener los índices que ordenarían un ndarray con numpy.argsort()
y_pred_ord = [...]

print('\nÍndices de los ejemplos a recomendar para el usuario {}, en función de sus valoraciones predichas:'.format(jj + 1))
print(y_pred_ord)

# Reentrenar incorporando nuevas valoraciones

Para reentrenar el modelo incorporando nuevas valoraciones de los usuarios, sólo hay que modificar la *Y* inicial con las nuevas valoraciones y marcar con un 1. la posición en la matriz *R*.

Sigue las instrucciones de la siguiente celda para incorporar nuevas valoraciones:

In [ ]:
# TODO: Incorpora 2 nuevas valoraciones de usuarios a 2 ejemplos a tu elección

# Escoge un índice de usuario y de ejemplo
i_1 = 2
k_1 = 2
i_2 = 3
k_3 = 3

# Escoge una valoración. Habitualmente toman valores entre [0, 2)
y[...] = 1.    
y[...] = 1.

# Márcalas como nuevas valoraciones en R
r[...] = 1.
r[...] = 1.

Ahora reentrena el modelo reejecutando la celda de entrenamiento y las siguientes hasta la celda anterior.

Comprueba cómo dichas posiciones muestran ahora la nueva valoración y no una predicción del modelo.

# Encontrar ejemplos y usuarios similares

Para encontrar la similitud entre 2 elementos, podemos computar la distancia euclídea entre ambos.

La distancia euclídea en este espacio n-dimensional representará la diferencia acumulada entre los coeficientes de dichos elementos, al igual que una distancia en un plano 2D o 3D es la diferencia acumulada entre las coordenadas de dichos puntos.

Encuentra ejemplos y usuarios similares siguiendo las instrucciones de la siguiente celda:

In [ ]:
# TODO: Encuentra ejemplos y usuarios similares entre sí

# Calcula la similaridad entre los 4 primeros ejemplos (X)
dist_ej = distance.cdist([...])

print('Similariad entre los 4 primeros ejemplos:')
print(dist_ej)

# Calcula la similaridad entre los 4 primeros usuarios (Theta)
dist_us = distance.cdist([...])

print('Similariad entre los 4 primeros usuarios:')
print(dist_us)

# Calcula el ejemplo más similar al primero
index_ej_similar = [...]
ej_similar = [...]

print('Coeficientes del ejemplo nº {} para los 5 primeros usuarios:'.format(0 + 1))
print(x[0, :5])
print('El ejemplo más similar al nº {} es el ejemplo nº {}'.format(0 + 1, index_ej_similar))
print('Coeficientes del ejemplo nº {} para los 5 primeros usuarios:'.format(index_ej_similar))
print(ej_similar[:5])

# Calcula el usuario más similar al primero
index_us_similar = [...]
us_similar = [...]

print('Coeficientes del usuario nº {} para los 5 primeros ejemplos:'.format(0 + 1))
print(theta[0, :5])
print('El usuario más similar al nº {} es el usuario nº {}'.format(0 + 1, index_us_similar))
print('Coeficientes del usuario nº {} para los 5 primeros ejemplos:'.format(index_us_similar))
print(us_similar[:5])

## Bonus: Comprobar qué sucede si no disponemos de suficientes valoraciones iniciales

*¿Qué sucede si no tenemos un nº mínimo de valoraciones inicialmente? ¿Y si hay algún ejemplo que no cuenta con ninguna valoración de ningún usuario, o un usuario que no ha valorado ningún ejemplo?*

*¿Crees que, en ese caso, podríamos entrenar el modelo y obtener resultados para dichos ejemplos y usuarios?*

Para comprobarlo, puedes p. ej. disminuir el porcentaje de valoraciones iniciales hasta un valor demasiado bajo, p. ej. un 10%, y comprobar qué sucede con la evolución de la función de coste del entrenamiento.